# Battery Model Declaration

In [1]:
using JuMP
using Gurobi
using DataFrames
using DataArrays

In [2]:
m = Model(solver=GurobiSolver())

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver set to Gurobi

# Parameter Declarations

## General Model Parameter Declarations 

In [3]:
iGenLoad = readtable("BatteryModel_GenLoad.csv",header=true)
iPrices = readtable("BatteryModel_Prices.csv",header=true)
iParameters = readtable("BatteryModel_Parameters.csv",header=true)

T = length(iPrices[:Hour]) 
pHours = iPrices[:Hour][1:T]


pBuyEnergy = iPrices[:pBuy_Energy][1:T]
pSellEnergy = iPrices[:pSell_Energy][1:T]
pNonControllableLoad = iGenLoad[:pNonControllableLoad][1:T]

pDelta = 1 #duration of time period (hr)
pCCurtail = 0.25 #cost of curtailment - will depend on the scenario ($/KWH)


0.25

## Generation Parameters

In [4]:
pGenAssets = [1 5 2.5 2.5] # this is where you specify the technology choices; will ultimately take this from dataframes
#genAssets = [pPVbank pBatt_NominalE pBatt_DischargeCapacity pBatt_ChargeCapacity]

1x4 Array{Float64,2}:
 1.0  5.0  2.5  2.5

### Non-Controllable Gen Parameters

In [5]:
pPV_Capacity = pGenAssets[1] #this is the total capacity of the PV panels 
pPV_Generation = pGenAssets[1]*iGenLoad[:pPV_Generation][1:T]

# declaring non-PV based non-controllable generation
pOtherNonControllableGen = iGenLoad[:pOtherNonControllableGen][1:T]

#total non-controllable generation = PV + other
@defExpr(pTotalNonControllableGen[t=1:T], pPV_Generation[t]+pOtherNonControllableGen[t])


### Battery Parameters

In [6]:
pBattNominalE = pGenAssets[2]
pBattDischargeCapacity = pGenAssets[3] #installed battery discharge power capacity (KW)
pBattChargeCapacity = pGenAssets[4] #installed battery charge power capacity (KW)

#pBatt_DischargeEff Efficiency of battery discharge
#pBatt_ChargeEff Efficiency of battery charge
#pBatt_CDeg battery degradation cost ($) #SB!!! Seems an odd thing to specify. Maybe base it off of cycle life, etc.
#pBatt_InitialSOC initial state of charge 
#pBatt_SOCMax maximum SOC
#pBatt_SOCMin minimum SOC

pBattDischargeEff = iParameters[1,:pBatt_DischargeEff]
pBattChargeEff = iParameters[1,:pBatt_ChargeEff]
pBattCDeg = iParameters[1,:pBatt_CDeg]
pBattInitialSOC = iParameters[1,:pBatt_InitialSOC]
pBattSOCMax = iParameters[1,:pBatt_SOCMax]
pBattSOCMin = iParameters[1,:pBatt_SOCMin]

0.1

# Variable Declarations 

## Power consumption variables 

In [7]:
@defVar(m, vPowerCurtail[t=1:T] >= 0) #curtailed power
@defVar(m, vPowerConsumed[t=1:T] >=0) # consumed power
@defVar(m, vPowerImportorExport[t=1:T], Bin)

@defVar(m, vPowerPurchased[t=1:T] >=0) # power ultimately purchased from utility

vPowerPurchased[t] >= 0 for all t in {1,2..23,24}

## Battery variables

In [8]:
@defVar(m, vBattSOC[t=1:T] >= 0) # SOC of battery ----- SB!!! Maybe make this a state variable, not a decision variable.
@defVar(m, vBattSOH[t=1:T] >= 0) #State of health of battery (degradation costs)
@defVar(m, vBattCharge[t=1:T] >= 0) #battery charging power
@defVar(m, vBattDischarge[t=1:T] >= 0) #battery discharging power
# @defVar(m, vBatt_Demand[t=1:T] >= 0) #load SB!!! confused by this for now
@defVar(m, vBattCorD[t=1:T], Bin) #batt charging or discharging binary variable. 
# @defVar(m, Bbd[t=1:T], Bin) #batt discharge - unclear if we need this.


vBattCorD[t] in {0,1} for all t in {1,2..23,24}

# Constraint Declarations

##Demand Balance

In [13]:
for t=1:T
    @addConstraint(m, vPowerConsumed[t] == pNonControllableLoad[t] + vBattCharge[t]) # + vScheduledLoads[t] + ThermalLoads[t] would normally include flexible load
end

@defExpr(sPowerProduced[t=1:T], pTotalNonControllableGen[t] + vBattDischarge[t])

@defExpr(sPowerExport[t=1:T], vPowerConsumed[t]-sPowerProduced[t])
 

#for t=1:T
#    if (pTotalNonControllableGen[t] + pBatt_NominalE) > 0
#        @addConstraint(m, sPowerExport[t] == sPowerProduced[t]-vPowerConsumed[t])
#    else 
#        @addConstraint(m, sPowerExport[t] == 0)
#    end
#end

#for t=1:T
@defExpr(sPowerPurchase[t=1:T], vPowerConsumed[t]-sPowerProduced[t])
#end

for t=1:T
    @addConstraint(m, sPowerExport[t]+sPowerPurchase[t] == vPowerConsumed[t]+sPowerProduced[t])
end

for t=1:T
    @addConstraint(m, sPowerExport[t] <= 10000000*(1-vPowerImportorExport[t]))
    @addConstraint(m, sPowerPurchase[t] <= 10000000*(vPowerImportorExport[t]))
end


## Battery Constraints

In [14]:
#constraining state of charge to appropriate limits
@addConstraint(m, vBattSOC[1] == pBattInitialSOC)

for t = 1:T
    @addConstraint(m, vBattSOC[t] <= pBattSOCMax)
    @addConstraint(m, vBattSOC[t] >= pBattSOCMin)
end 

# defining charging losses and efficiency
@defExpr(sBattDischargeLosses[t=1:T], pBattDischargeEff*vBattDischarge[t])
@defExpr(sBattChargeLosses[t=1:T], pBattChargeEff*vBattCharge[t])
@defExpr(sBattLosses[t=1:T], sBattChargeLosses[t] + sBattDischargeLosses[t])

@defExpr(sBattOutput[t=1:T], vBattDischarge[t] - sBattDischargeLosses[t])
@defExpr(sBattInput[t=1:T], vBattCharge[t] + sBattChargeLosses[t])
        

if pBattNominalE > 0
    for t = 2:T  
        @addConstraint(m, vBattSOC[t-1] - (pDelta/pBattNominalE)*(sBattOutput[t-1] - sBattInput[t-1] + sBattLosses[t-1]) == vBattSOC[t])
    end 
else 
    for t = 2:T
        @addConstraint(m, vBattSOC[t] == vBattSOC[1])
    end 
end 

#define charge or discharge constraint
for t=1:T
    @addConstraint(m, vBattCharge[t] <= 1000000*vBattCorD[t])
    @addConstraint(m, vBattDischarge[t] <= 1000000*(1-vBattCorD[t]))
end



In [15]:
@defExpr(TotalCost,sum{pDelta*pBuyEnergy[t]*sPowerPurchase[t] + pDelta*pSellEnergy[t]*sPowerExport[t],t=1:T})

0.2 vPowerConsumed[1] - 0.2 vBattDischarge[1] + 0.2 vPowerConsumed[2] - 0.2 vBattDischarge[2] + 0.2 vPowerConsumed[3] - 0.2 vBattDischarge[3] + 0.2 vPowerConsumed[4] - 0.2 vBattDischarge[4] + 0.2 vPowerConsumed[5] - 0.2 vBattDischarge[5] + 0.2 vPowerConsumed[6] - 0.2 vBattDischarge[6] + 0.2 vPowerConsumed[7] - 0.2 vBattDischarge[7] + 0.2 vPowerConsumed[8] - 0.2 vBattDischarge[8] + 0.2 vPowerConsumed[9] - 0.2 vBattDischarge[9] + 0.2 vPowerConsumed[10] - 0.2 vBattDischarge[10] + 0.2 vPowerConsumed[11] - 0.2 vBattDischarge[11] + 0.2 vPowerConsumed[12] - 0.2 vBattDischarge[12] + 0.2 vPowerConsumed[13] - 0.2 vBattDischarge[13] + 0.2 vPowerConsumed[14] - 0.2 vBattDischarge[14] + 0.2 vPowerConsumed[15] - 0.2 vBattDischarge[15] + 0.2 vPowerConsumed[16] - 0.2 vBattDischarge[16] + 0.2 vPowerConsumed[17] - 0.2 vBattDischarge[17] + 0.6 vPowerConsumed[18] - 0.6 vBattDischarge[18] + 0.6 vPowerConsumed[19] - 0.6 vBattDischarge[19] + 0.6 vPowerConsumed[20] - 0.6 vBattDischarge[20] + 0.6 vPowerConsumed

In [16]:
@setObjective(m, Min, TotalCost)
# + PowerImport*PowerBuyPrice(t) + PowerExport*PowerSellPrice(t) + Fixed(t) )

0.2 vPowerConsumed[1] - 0.2 vBattDischarge[1] + 0.2 vPowerConsumed[2] - 0.2 vBattDischarge[2] + 0.2 vPowerConsumed[3] - 0.2 vBattDischarge[3] + 0.2 vPowerConsumed[4] - 0.2 vBattDischarge[4] + 0.2 vPowerConsumed[5] - 0.2 vBattDischarge[5] + 0.2 vPowerConsumed[6] - 0.2 vBattDischarge[6] + 0.2 vPowerConsumed[7] - 0.2 vBattDischarge[7] + 0.2 vPowerConsumed[8] - 0.2 vBattDischarge[8] + 0.2 vPowerConsumed[9] - 0.2 vBattDischarge[9] + 0.2 vPowerConsumed[10] - 0.2 vBattDischarge[10] + 0.2 vPowerConsumed[11] - 0.2 vBattDischarge[11] + 0.2 vPowerConsumed[12] - 0.2 vBattDischarge[12] + 0.2 vPowerConsumed[13] - 0.2 vBattDischarge[13] + 0.2 vPowerConsumed[14] - 0.2 vBattDischarge[14] + 0.2 vPowerConsumed[15] - 0.2 vBattDischarge[15] + 0.2 vPowerConsumed[16] - 0.2 vBattDischarge[16] + 0.2 vPowerConsumed[17] - 0.2 vBattDischarge[17] + 0.6 vPowerConsumed[18] - 0.6 vBattDischarge[18] + 0.6 vPowerConsumed[19] - 0.6 vBattDischarge[19] + 0.6 vPowerConsumed[20] - 0.6 vBattDischarge[20] + 0.6 vPowerConsumed

In [17]:
solve(m)

Optimize a model with 456 rows, 216 columns and 1002 nonzeros
Coefficient statistics:
  Matrix range    [2e-01, 1e+07]
  Objective range [2e-01, 6e-01]
  Bounds range    [1e+00, 1e+00]
  RHS range       [8e-06, 1e+07]
Presolve removed 110 rows and 79 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available processors)

Model is infeasible or unbounded
Best objective -, best bound -, gap -


:InfeasibleOrUnbounded